In [23]:
import itertools as its
import math as m 
import sys

def CreateBlockMatrix(dimensions, blockshape, blocklist):
    '''Create one matrix out of several specified blocks'''
    return matrix.block(dimensions[0],dimensions[1], [blocklist[t] for t in blockshape])



def is_zeroone(matrix):
    '''Check if a matrix is a zero-one matrix'''
    for row in matrix:
        for entry in row:
            if entry not in {0,1}:
                return False

    return True

def switchingblock(level, blocknumber, sort):
    '''Create a regular orthogonal permutation-based matrix of a certain level, number of blocks and sort'''
    J = ones_matrix(level)
    O = zero_matrix(level)
    Y = level*identity_matrix(level)- J

    def rotate(l):
        return [l[-1]] + l[:-1]

    baserow = [0,2] + [1]*(blocknumber-2)
    shapematrix = []
    for _ in range(blocknumber):
        shapematrix = shapematrix + baserow
        baserow = rotate(baserow)


    if sort == 'wqh':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [Y,O,J])/level
    if sort == 'gm':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [-Y,O,J])/level

def getvertices(n, S):
    '''Gives a list where the numbers in S are moved to the front, which will be used as a permutation'''
    complement = set(range(1,n+1)) - set(S)
    return S + list(complement)

def symmetrymatrices(n, blocks, level, sort, recursionlevel):
    '''This function recursively finds all permutations (as matrices) within the switching set.
    At each recursion level it takes any set of level size and puts it to the front. We do this, because the switching matrix has a symmetry group induced by rotating the blocks and rotating within the blocks. Therefore it only matters what the set of vertices in each block are, so we pick combinations. First we pick the second to last block, then at each recursion level we go back until we pick the first block.For each block we may only pick from [1,....,blocks*level-1]. Thus ensuring that the last block has the highest number in it. This beats the symmetry of the block rotation.'''

    #I could remove n and replace by blocks*level just have to add identity matrix to P1
    if blocks == 2 and sort == 'gm':
        return [identity_matrix(n)]
    if recursionlevel == 1:
        return [identity_matrix(n)]

    matrices= []
    for X in Combinations(range(level*(blocks-recursionlevel)+1, blocks*level), level):
        Xmatrix = Permutation(getvertices(n, X)).to_matrix()
        for P in symmetrymatrices(n, blocks, level, sort, recursionlevel-1):
            # As the matrices will be transposed (on the left) in bruteswitching. The way to ensure that the highest level does the second to last block is to multiply Xmatrix on the left. To be honest I don't think it matters, you will get the same set of symmetrymatrices either way, but at least this way I can explain it.
            matrices.append(Xmatrix*P)

    return matrices

def symmetryreduce(list_of_edges, matrix):
    Aut = Graph(matrix).automorphism_group()

    list_of_representatives = []
    set_of_edges_in_orbits = []
    for M in list_of_edges:
        if M not in set_of_edges_in_orbits:
            list_of_representatives.append(M)
            for P in Aut:
                set_of_edges_in_orbits.append(M*Permutation(P).to_matrix())
    
    return list_of_representatives

    
def BuildGraphsWithSwitching(outersize, switchingsize, outergraph, switchingsets, valid_adjacencies, first_column):
    allgraphswithswitching = []
    if outersize > 0:
        allvalid_edges = [matrix(outersize-1, switchingsize, M) for M in its.product(valid_adjacencies, repeat=outersize-1)]
        b = len(allvalid_edges)
    else:
        return switchingsets
    
    for A in switchingsets:
        for W in first_column:
            for V in allvalid_edges:
                V = V.stack(matrix(1, switchingsize, W))
                switchingmatrix = A.augment(V.transpose())
                for C in outergraph:
                    bottommatrix= V.augment(C)
                    allgraphswithswitching.append(switchingmatrix.stack(bottommatrix))
        
    return allgraphswithswitching

def reducelisttoiso(graphlist):
    immutablegraphs = [G.copy(immutable=True) for G in graphlist]
    return set(immutablegraphs)

        
def checkcospectralmate(Q, graphswithswitching):
    hascospectralmate = []
    for A in graphswithswitching:
        Bcanlabel =  Graph(Q.transpose()*A*Q).canonical_label()
        Acanlabel = Graph(A).canonical_label()
        if Bcanlabel != Acanlabel:
            G = Acanlabel.complement()
            H = Bcanlabel.complement()
            hascospectralmate += [Acanlabel, Bcanlabel, G.canonical_label(), H.canonical_label()]
    
    return reducelisttoiso(hascospectralmate)

def Genswitchingset(level, blocks, sort):
    '''Create all labelled graphs (up to complementation) that form a switching set for the switching with given blocks, level and sort 
        up to permutation that leaves Q fixed.'''
    switchingset = []

    Q = switchingblock(level, blocks, sort)
    Permlist = symmetrymatrices(blocks*level, blocks, level, sort, blocks)
    n = level*blocks
    for G in graphs.nauty_geng(str(n) + ' 0:' + str(m.floor(n*(n-1)/4))):
        A = G.adjacency_matrix()
        for P in Permlist:
            B = Q.transpose()*P.transpose()*A*P*Q
            if is_zeroone(B):
                switchingset.append(P.transpose()*A*P)

    return [k for k, _ in its.groupby(sorted(switchingset))]

def GoodEdges(level, number_of_blocks, sort):
    '''Create a set of vectors that satisfy the conditions for a permutation family switching'''
    goodedges = []
    for v in its.product(range(2), repeat = level*number_of_blocks):
        sumsperblock = []
        for i in range(number_of_blocks):
            sumsperblock.append(sum(v[i*level:level*(i+1)]) % level)

        if sort == 'wqh':        
            if len(set(sumsperblock)) == 1:
                goodedges.append(v)

        elif sort == 'gm':
            should_append = True
            for i in range(number_of_blocks):
                if (sumsperblock[i] + sumsperblock[(i+1) % number_of_blocks]) % level != 0:
                    should_append = False

            if should_append:
                goodedges.append(v)

    return goodedges

def Qresp(Q,m):
    qrespvectors = []
    for vec in its.product(range(2), repeat = m):
        if all(i in {0,1} for i in vector(vec)*Q):
            qrespvectors.append(vec)
    
    return qrespvectors
############################################

def BuildandEnumerateCospectralgraphs(level, numberblocks, sort, added_vertices):
    goodedges = GoodEdges(level, numberblocks, sort)
    switchingsets = Genswitchingset(level, numberblocks, sort)

        
    reduceswitchcolumn = []
    firstandswitched = []
    Q = switchingblock(level, numberblocks, sort)
    for x in goodedges:
        y = vector(x)
        if y not in firstandswitched:
            firstandswitched.append(y)
            firstandswitched.append(y*Q)
            reduceswitchcolumn.append(y)

    HasAH = BuildGraphsWithSwitching(added_vertices, numberblocks*level, [G.adjacency_matrix() for G in graphs(added_vertices)], switchingsets, goodedges, reduceswitchcolumn)
  
    Q = switchingblock(level, numberblocks, sort)
    Q = Q.block_sum(identity_matrix(added_vertices))

    return checkcospectralmate(Q, HasAH)

def BuildandEnumerateGeneral(Q, added_vertices, switchingsets, parts, reduceswitchcolumn = None):
    m = Q.dimensions()[0]
    goodedges = Qresp(Q, m)
    if reduceswitchcolumn is None:
        reduceswitchcolumn = goodedges

    Allgrapswithswitching = BuildGraphsWithSwitching(added_vertices, m, [G.adjacency_matrix() for G in graphs(added_vertices)], switchingsets, goodedges, reduceswitchcolumn)
    Q = Q.block_sum(identity_matrix(added_vertices))

    return checkcospectralmate(Q, Allgrapswithswitching)

def BuildandEnumerateHardcoded(level, numberblocks, sort, added_vertices, switchingsets, reduceswitchcolumn):
    goodedges = GoodEdges(level, numberblocks, sort)


    HasAH = BuildGraphsWithSwitching(added_vertices, numberblocks*level, [G.adjacency_matrix() for G in graphs(added_vertices)], switchingsets, goodedges, reduceswitchcolumn)
    Q = switchingblock(level, numberblocks, sort)
    Q = Q.block_sum(identity_matrix(added_vertices))

    return checkcospectralmate(Q, HasAH)

def export(filename, object_list, method_name):
    with open(filename, 'w') as f:
        for obj in object_list:
            f.write(attrcall(method_name)(obj) + '\n')

##################################################




OLD VERSION


In [ ]:

def is_zeroone(matrix):
    '''Check if a matrix is a zero-one matrix'''
    for row in matrix:
        for entry in row:
            if entry not in {0,1}:
                return False

    return True

def orbitQinf(n, blocks, level, sort, recursionlevel):
    '''This function recursively finds all permutations (as matrices) that do not fix the switching matrix Q of the infinite family.
    At each recursion level it takes any set of size equal to the level (containing numbers that we not put to the front before) and puts it at the front of the permutation. 
    The matrix Q has a symmetry group induced by rotating the blocks and any permutation within the blocks. 
    Therefore it only matters what the set of vertices in each block are, so we pick combinations 
    and by picking from [1,....,blocks*level-1] we ensure that the last block contains the highest number, so we deal with the symmetry of the block rotation.'''

    #I could remove n and replace by blocks*level just have to add identity matrix to P1
    if blocks == 2 and sort == 'gm':
        return [identity_matrix(n)]
    if recursionlevel == 1:
        return [identity_matrix(n)]

    def getvertices(n, S):
        #pulls vertices in S to the front of the list.
        complement = set(range(1,n+1)) - set(S)
        return S + list(complement)

    matrices= []
    for X in Combinations(range(level*(blocks-recursionlevel)+1, blocks*level), level):
        Xmatrix = Permutation(getvertices(n, X)).to_matrix()
        for P in symmetrymatrices(n, blocks, level, sort, recursionlevel-1):
            # As the matrices will be transposed (on the left) in bruteswitching. 
            # The way to ensure that the highest level does the second to last block is to multiply Xmatrix on the left. 
            matrices.append(Xmatrix*P)

    return matrices

def Genswitchingset(level, blocks, sort):
    '''Create all Gamma that are a switching graph (up to complementation) for a Q from the infinite family. 
    By simply going through all graphs.
    ..............'''
    switchingset = []

    Q = switchingmatrixinf(level, blocks, sort)
    Permlist = orbitQinf(blocks*level, blocks, level, sort, blocks)
    n = level*blocks
    for G in graphs.nauty_geng(str(n) + ' 0:' + str(m.floor(n*(n-1)/4))):
        A = G.adjacency_matrix()
        for P in Permlist:
            B = Q.transpose()*P.transpose()*A*P*Q
            if is_zeroone(B):
                switchingset.append(P.transpose()*A*P)

    return [k for k, _ in its.groupby(sorted(switchingset))]

def GoodEdges(level, number_of_blocks, sort):
    '''Create the set of Q-respecting vectors for a Q from the infinite family.'''

    goodedges = []
    for v in its.product(range(2), repeat = level*number_of_blocks):
        sumsperblock = []
        for i in range(number_of_blocks):
            sumsperblock.append(sum(v[i*level:level*(i+1)]) % level)

        if sort == 1:        
            if len(set(sumsperblock)) == 1:
                goodedges.append(v)

        elif sort == -1:
            should_append = True
            for i in range(number_of_blocks):
                if (sumsperblock[i] + sumsperblock[(i+1) % number_of_blocks]) % level != 0:
                    should_append = False

            if should_append:
                goodedges.append(v)

    return goodedges

################################################
def BuildandEnumerateCospectralgraphs(level, numberblocks, sort, added_vertices):
    goodedges = GoodEdges(level, numberblocks, sort)
    switchingsets = Genswitchingset(level, numberblocks, sort)

        
    reduceswitchcolumn = []
    firstandswitched = []
    Q = switchingmatrixinf(level, numberblocks, sort)
    #------------------------------- dit deel met reduceswitchcolumn mag schoner
    for x in goodedges:
        y = vector(x)
        if y not in firstandswitched:
            firstandswitched.append(y)
            firstandswitched.append(y*Q)
            reduceswitchcolumn.append(y)

    HasQswitching = BuildGraphsWithSwitching(added_vertices, numberblocks*level, switchingsets, goodedges, reduceswitchcolumn)
  
    Q = switchingmatrixinf(level, numberblocks, sort)
    Q = Q.block_sum(identity_matrix(added_vertices))

    return checkcospectralmate(Q, HasQswitching)

In [36]:
first = [vector([0,0,0,0,0,0]), vector([1,1,1,1,1,1]), vector([1,1,0,0,0,0]), vector([1,1,1,1,0,0]), vector([1,0,1,0,1,0]), vector([0,1,0,1,0,1]), vector([0,1,1,0,1,0]), vector([0,1,0,1,1,0])]
O = zero_matrix(2)
N = matrix([[0,0],[1,1]])

irredmatrix = CreateBlockMatrix((3,3), [0,1,2,2,0,1,1,2,0], [O, N, N.transpose()])

# def rotate(l):
#     return [l[-1]] + l[:-1]

# baserow = [-1,1,1,0,1,0,0]
# switchingmat = []
# for _ in range(7):
#         switchingmat.append(baserow)
#         baserow = rotate(baserow)
# Q = 1/2*matrix(switchingmat)

# special = matrix([[0,0,0,0,0,1,1], [0,0,0,0,0,0,1], [0,0,0,1,0,1,1], [0,0,0,0,1,0,1], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1], [0,0,0,0,0,0,0]])
# special = special + special.T
# irredmatrix = [graphs.CycleGraph(7).adjacency_matrix(), special]
Graphswithcospectralmates = BuildandEnumerateCospectralgraphs(3, 2, 'wqh', 3) 



export('lvl2switching/graphs_with_cospectralmates/enumgm6plus3.g6', Graphswithcospectralmates, 'graph6_string')